cnn code with tensorflow , 96.1% accuracy , python2.7, tensorflow1.1

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from skimage import io, transform
import os, gc, sys, glob
import pandas as pd
import numpy as np
from tqdm import tqdm
def read_img(img_path):
    img = io.imread(img_path)
    img = transform.resize(img, (128, 128))
    return img
train_set = pd.read_csv('images/train_labels.csv/train_labels.csv')
train_img, test_img = [], []
for img_path in tqdm(train_set['name'].iloc[: ]):
    train_img.append(read_img('images/train/' + str(img_path) + '.jpg'))
train_img = np.array(train_img, np.float32)
train_label = np.array(train_set['invasive'].iloc[: ])
train_label = train_label.reshape([-1,1])
val_split_num = int(round(0.2*len(train_label)))
x_train = train_img[val_split_num:]
y_train = train_label[val_split_num:]
x_test = train_img[:val_split_num]
y_test = train_label[:val_split_num]
evaluation_data = x_test, y_test
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
            # featurewise_center = True,
            #rotation_range = 30,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            # zca_whitening = True,
            shear_range = 0.2,
            zoom_range = 0.2,
            horizontal_flip = True,
            vertical_flip = True,
            fill_mode = 'nearest')
datagen.fit(x_train)


In [ ]:
import tensorflow as tf
class network(object):
    def __init__(self):

        self.learning_rate = 0.001
        self.batch_size = 64
        self.nclasses = 1
        self.n_input = 128*128
        self.displaytime = 10
        self.epoches = 1000
        self.regularation_param = 1e-6
        self.weights = {
            'wc1': tf.Variable(initial_value=tf.random_uniform(shape=[3, 3, 3, 16], minval=-0.05, maxval=0.05)),
            'wc2': tf.Variable(initial_value=tf.random_uniform(shape=[3, 3, 16, 32], minval=-0.05, maxval=0.05)),
            'wc3': tf.Variable(initial_value=tf.random_uniform(shape=[3, 3, 32, 64], minval=-0.05, maxval=0.05)),
            'wc4': tf.Variable(initial_value=tf.random_uniform(shape=[3, 3, 64, 128], minval=-0.05, maxval=0.05)),
            'wd1': tf.Variable(initial_value=tf.random_uniform(shape=[12*12*128, 1024], minval=-0.05, maxval=0.05)),
            'wd2': tf.Variable(initial_value=tf.random_uniform(shape=[1024, 512], minval=-0.05, maxval=0.05)),
            # OOM
            'out': tf.Variable(initial_value=tf.random_normal(shape=[512, self.nclasses]))

        }
        self.biases = {
            'b1': tf.Variable(initial_value=tf.zeros([16])),
            'b2': tf.Variable(initial_value=tf.zeros([32])),
            'b3': tf.Variable(initial_value=tf.zeros([64])),
            'b4': tf.Variable(initial_value=tf.zeros([128])),
            'bfc1': tf.Variable(initial_value=tf.zeros([1024])),
            'bfc2': tf.Variable(initial_value=tf.zeros([512])),
            'out': tf.Variable(initial_value=tf.zeros([self.nclasses]))
        }
        self.x = tf.placeholder(tf.float32, [None, 128, 128, 3])
        self.y = tf.placeholder(tf.float32, [None, self.nclasses])
        self.keep_prob_1 = tf.placeholder(tf.float32)  # dropout (keep probability)
        self.keep_prob_2 = tf.placeholder(tf.float32)  # dropout (keep probability)


    def conv2d(self, x, W, b, strides=1):
        output = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')
        output = tf.nn.bias_add(output, b)
        return tf.nn.relu(output)

    def maxpool2d(self, x, k=2, strides=2):
        output = tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, strides, strides, 1], padding='VALID')
        return output

    def convNet(self, x):
        ## reshape x for conv layer
        #x = tf.reshape(x, shape=[-1, 28, 28, 1])
        conv1 = self.conv2d(x, self.weights['wc1'], self.biases['b1'])
        pool1 = self.maxpool2d(conv1)
        conv2 = self.conv2d(pool1, self.weights['wc2'], self.biases['b2'])
        pool2 = self.maxpool2d(conv2)
        conv3 = self.conv2d(pool2, self.weights['wc3'], self.biases['b3'])
        pool3 = self.maxpool2d(conv3)
        conv4 = self.conv2d(pool3, self.weights['wc4'], self.biases['b4'])
        ## reshape output for fully-connected layer
        fc1 = tf.reshape(conv4, [-1, self.weights['wd1'].get_shape().as_list()[0]])
        fc1 = tf.add(tf.matmul(fc1, self.weights['wd1']), self.biases['bfc1'])
        fc1 =  tf.nn.relu(fc1)
        fc1 = tf.nn.dropout(fc1, self.keep_prob_1)

        fc2 = tf.add(tf.matmul(fc1, self.weights['wd2']), self.biases['bfc2'])
        fc2 =  tf.nn.relu(fc2)
        fc2 = tf.nn.dropout(fc2, self.keep_prob_2)
        
        out = tf.add(tf.matmul(fc2, self.weights['out']), self.biases['out'])
        #out = tf.reshape([-1,])
        return out


    def cost(self, logits, labels):
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=labels)) \
     + self.regularation_param * (tf.nn.l2_loss(self.weights['wc1']) + tf.nn.l2_loss(self.weights['wc2']) + \
                                 tf.nn.l2_loss(self.weights['wc3']) + tf.nn.l2_loss(self.weights['wc4']) + \
                                 tf.nn.l2_loss(self.weights['wd1']) + tf.nn.l2_loss(self.weights['out']))

    def evaluation(self, pred, labels):
        
        correct_pred = tf.equal(tf.round(tf.reduce_max(tf.sigmoid(pred), 1)), tf.reduce_max(labels, 1))
        return tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
    def train(self, datagen, x_train, y_train, evaluation_data):
        pred = self.convNet(self.x)
        cost = self.cost(pred, labels=self.y)
        acc = self.evaluation(pred, self.y)
        tf.summary.scalar('cost', cost)
        optimizer = tf.train.MomentumOptimizer(learning_rate=self.learning_rate, momentum=0.8,use_nesterov=True).minimize(cost)
        init = tf.global_variables_initializer()
        merged = tf.summary.merge_all()
        with tf.Session() as sess:
            summary_writer = tf.summary.FileWriter('/home/gql/Documents/logs/', sess.graph)
            sess.run(init)
            for epoch in range(self.epoches):
                print "epoch:", epoch
                step = 0
                n = len(x_train)
                for batch_x, batch_y in datagen.flow(x_train, y_train, self.batch_size):
                    
                    # update mini_bacth
                    ## bp for mini_batch and update w,b
                    summary, _= sess.run([merged ,optimizer], feed_dict={self.x: batch_x, self.y: batch_y, self.keep_prob_1: 0.65, self.keep_prob_2: 0.55})
                    step += 1
                    summary_writer.add_summary(summary, step)
                    if step % self.displaytime == 0:
                        loss, accuracy = sess.run([cost, acc], feed_dict={self.x: batch_x, self.y: batch_y, self.keep_prob_1: 1., self.keep_prob_2: 1.})
                        print "Iter " + str(step * self.batch_size) + ", Minibatch Loss= " + \
                            "{:.6f}".format(loss) + ", Training Accuracy= " + \
                            "{:.5f}".format(accuracy) + ",Testing Accuracy= ",\
                            sess.run(acc, feed_dict={self.x: evaluation_data[0][:256],
                                              self.y: evaluation_data[1][:256],
                                              self.keep_prob_1: 1., self.keep_prob_2: 1.})
                    if step >= n / self.batch_size:
                        break
               
            print 'optimization finished!'
            print "Testing Accuracy:", \
                sess.run(acc, feed_dict={self.x: evaluation_data[0],
                                              self.y: evaluation_data[1],
                                              self.keep_prob_1: 1., self.keep_prob_2: 1.})
            print x_train.shape
            print "training Accuracy:", \
                sess.run(acc, feed_dict={self.x: x_train[:512],
                                              self.y: y_train[:512],
                                              self.keep_prob_1: 1., self.keep_prob_2: 1.})

In [ ]:
net = network()
net.train(datagen, x_train, y_train, evaluation_data)